## *Rodar notebook toda segunda-feira pra atualizar informações de transferências, treinadores e principalmente do clube*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]

print("Data:", agora)

Data: 2021-11-16 08:02:02


In [3]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Treinadores

url= 'https://sokker.org/xml/trainers.xml'
df_trainers= pega_dados(url, sessao, search= 'trainer')
df_trainers

# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_trainers= df_trainers.assign(date_extracted= agora)

# Soma de valores nulos e duplicados
nulos= df_trainers.isnull().sum().sum()
duplicados=  df_trainers.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados nos dados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_trainers.to_sql('trainers', con=db, if_exists='append', index= False)

    print('Dados extraídos com sucesso!')

In [4]:
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()
df = pd.read_sql_query("SELECT * FROM trainers", db)
df

,ID,date_extracted,name,surname,countryID,age,teamID,wage,signedContract,job,skillStamina,skillPace,skillTechnique,skillPassing,skillKeeper,skillDefending,skillPlaymaking,skillScoring,skillCoach
0,10206425,2021-07-27 14:46:51,Irapuan,Bilha,21,42,79085,55500,1,1,13,5,12,7,11,15,14,1,10
1,10206425,2021-07-31 09:50:48,Irapuan,Bilha,21,42,79085,55500,1,1,13,5,12,7,11,15,14,1,10
2,10207283,2021-07-31 09:50:48,Silnei,Bugs,21,64,79085,31800,1,2,12,16,7,1,3,7,1,9,7
3,10206425,2021-08-02 09:55:05,Irapuan,Bilha,21,42,79085,55500,1,1,13,5,12,7,11,15,14,1,10
4,10207283,2021-08-02 09:55:05,Silnei,Bugs,21,64,79085,31800,1,2,12,16,7,1,3,7,1,9,7
5,10206425,2021-08-09 11:32:44,Irapuan,Bilha,21,43,79085,55500,1,1,13,5,12,7,11,15,14,1,10
6,10207283,2021-08-09 11:32:44,Silnei,Bugs,21,65,79085,31800,1,2,12,16,7,1,3,7,1,9,7
7,10206425,2021-08-16 10:17:59,Irapuan,Bilha,21,43,79085,55500,1,1,13,5,12,7,11,15,14,1,10
8,10207283,2021-08-16 10:17:59,Silnei,Bugs,21,65,79085,31800,1,2,12,16,7,1,3,7,1,9,7
9,10206425,2021-08-23 11:08:41,Irapuan,Bilha,21,43,79085,55500,1,1,13,5,12,7,11,15,14,1,10


### Transferências

In [5]:
#Pega informações únicas das transferências já adicionadas para que não sejam inseridos valores duplicados na base
transfers = pd.read_sql_query("SELECT * FROM transfers", db)
dates= transfers['date'].unique()
ids= transfers['ID'].unique()

# Pegando informações das partidas retirando aquelas que já foram pegas
url= 'https://sokker.org/xml/transfers.xml'
df_transfers= pega_dados(url, sessao, search= 'transfer')


#Retirando valores nulos e duplicados 
query= 'date not in @dates and ID not in @ids'
df_transfers.dropna(inplace= True)
df_transfers= df_transfers.query(query)
df_transfers

,ID,sellerTeamID,buyerTeamID,sellerTeamName,buyerTeamName,playerID,date,price,playerValue


In [6]:
if len(df_transfers) > 0:
    
    # Adiciona dados à tabela
    df_transfers.to_sql('transfers', con=db, if_exists='append', index= False)

In [7]:
transfers = pd.read_sql_query("SELECT * FROM transfers", db)
transfers

,ID,sellerTeamID,buyerTeamID,sellerTeamName,buyerTeamName,playerID,date,price,playerValue
0,6942634,28049,79085,PlastovFC,Organized Chaos FC,37691056,2021-07-27 17:44:15,4,504000
1,6942606,52642,79085,Klucha Team,Organized Chaos FC,37634685,2021-07-27 16:44:07,200000,553000
2,6942327,95215,79085,ks.śląsk,Organized Chaos FC,37690980,2021-07-26 16:24:46,16000,471000
3,6942168,43275,79085,Chiles Mexicanos,Organized Chaos FC,37937282,2021-07-25 22:15:10,6000,273000
4,6942114,50992,79085,Unia Oświęcim 97,Organized Chaos FC,37432229,2021-07-25 21:20:21,1,745000
5,6941058,71211,79085,fcm juventus,Organized Chaos FC,37008614,2021-07-23 14:18:53,55000,681000
6,6940834,52314,79085,F.C. Roztocza,Organized Chaos FC,37276405,2021-07-22 16:00:43,100000,972000
7,6940612,112375,79085,Arsenal Fc.,Organized Chaos FC,36103236,2021-07-21 17:40:45,18000,1651000
8,6940546,46083,79085,Urunal FC,Organized Chaos FC,36519432,2021-07-21 12:34:59,13,2701000
9,6940428,11578,79085,Oxford United F.C.,Organized Chaos FC,37262806,2021-07-20 22:42:31,7,686000


In [8]:
len(transfers)

29

### Clube

In [9]:
url= f'https://sokker.org/xml/team-{club_id}.xml'
df_team= pega_dados(url, sessao, search= 'team')

# Inserindo a data de hoje que será importante para acompanharmos evolução do clube
df_team= df_team.assign(date_extracted= agora)

# Retirando colunas que não serão guardadas na base
df_team.drop(columns= ['colShirtKeep', 'colTrausKeep', 'colShirt', 
                       'colTraus', 'colShirt2', 'colTraus2'], inplace= True)

In [10]:
if len(df_team) > 0:
    
    # Adiciona dados à tabela
    df_team.to_sql('teams', con=db, if_exists='append', index= False)

In [11]:
teams = pd.read_sql_query("SELECT * FROM teams", db)
teams

,teamID,name,countryID,regionID,dateCreated,rank,national,arenaName,money,trainingType,trainingFormation,fanclubCount,fanclubMood,juniorsMax,date_extracted
0,79085,Organized Chaos FC,21,381,2021-07-16 01:49,1637.12,0,Organized Chaos FC Arena,625431,6,2,232,6,0,2021-07-27 14:46:51
1,79085,Organized Chaos FC,21,381,2021-07-16 01:49,1612.54,0,Organized Chaos FC Arena,97346,6,1,232,5,0,2021-07-31 09:50:48
2,79085,Organized Chaos FC,21,381,2021-07-16 01:49,1748.92,0,Organized Chaos FC Arena,126021,6,1,261,6,0,2021-08-02 09:55:05
3,79085,Organized Chaos FC,21,381,2021-07-16 01:49,1830.62,0,Organized Chaos FC Arena,594557,6,1,296,6,0,2021-08-09 11:32:44
4,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2083.78,0,Organized Chaos FC Arena,97756,6,1,333,6,0,2021-08-16 10:17:59
5,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2139.21,0,Organized Chaos FC Arena,322065,6,1,365,6,0,2021-08-23 11:08:41
6,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2202.23,0,Organized Chaos FC Arena,297141,6,1,396,6,0,2021-08-30 09:40:24
7,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2335.95,0,Organized Chaos FC Arena,326537,6,1,428,6,0,2021-09-06 12:57:41
8,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2469.79,0,Organized Chaos FC Arena,470051,8,1,458,6,0,2021-09-13 09:54:34
9,79085,Organized Chaos FC,21,381,2021-07-16 01:49,2485.23,0,Organized Chaos FC Arena,518301,8,1,487,6,0,2021-09-22 18:58:01


In [12]:
len(teams)

18

In [13]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()